In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [2]:
persist_directory = "/Users/dardonacci/Documents/dardonacci/2-Code/metabrain/vector_stores/hummingbot/chroma/"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_scripts",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [6]:
from useful_prompts.create_scripts import prompt_1

prompt_1

"\n# Algorithmic Trading Strategy Development in Hummingbot\n\nAs an expert in bot trading, your assignment involves creating a comprehensive Python script for a trading strategy using Hummingbot, a community-driven open source project designed to aid in building high-frequency trading bots. \n\nKeep the following guidelines in mind during your development phase:\n\n- The Python script should be written with object-oriented principles in mind. \n- Class variables should be used to configure your script accordingly.\n- Your script will be extending the `ScriptStrategyBase` class, effectively inheriting all its methods and attributes.\n- Each controller should be initialized while defining your class variables.\n- Your executor handler should commence within your `__init__` method.\n\nBe as thorough as possible, detailing your thought process and the purpose of each segment of code. Should you require additional information from the user, don't hesitate to ask for clarification. However,

In [7]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Build prompt

prompt = ChatPromptTemplate.from_template(template=prompt_1)
model = ChatOpenAI(model_name=llm_name, temperature=0)
output_parser = StrOutputParser()

chain = prompt | model

In [8]:
query = """I would like to create a Directional Strategy using Hummingbot using the MACDBB controller. Can you the script for me and suggest values for the indicators?
"""

In [9]:
chain.invoke(
    input={
        "question": query,
    }
)

KeyError: 'context'

In [10]:
from langchain.schema.runnable import RunnableMap

retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke(
    input={
        "question": query,
    }
)

In [ ]:
class Strategy(BaseModel):
    """Strategy created by the LLM for the user."""
    code: str = Field(description="Provide the python code for the strategy.")
    summary: str = Field(description="Provide a concise summary of what the strategy is doing.")

In [ ]:

strategy_extraction_function = [
    convert_pydantic_to_openai_function(Strategy)
]
tagging_model = model.bind(
    functions=strategy_extraction_function,
    function_call={"name":"Strategy"}
)

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | StrOutputParser() | tagging_model | JsonOutputFunctionsParser()

In [ ]:
result = tagging_chain.invoke(
    input={
        "question": query,
    }
)

In [ ]:
print(result["summary"])

In [ ]:
print(result["code"])